In [2]:
!nvidia-smi

Sun Oct 20 20:03:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti     Off |   00000000:01:00.0  On |                  N/A |
| N/A   41C    P5             13W /   80W |      83MiB /   6144MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
import subprocess

In [4]:
PROJECT_ROOT = "/home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-"
DARKNET_PATH = os.path.join(PROJECT_ROOT, "darknet")
DATA_PATH = os.path.join(PROJECT_ROOT, "data", "processed_data")

In [5]:
# 2.1 Create train.txt and val.txt
def create_image_list_file(set_type):
    image_dir = os.path.join(DATA_PATH, set_type, "images")
    output_file = os.path.join(DATA_PATH, f"{set_type}.txt")
    
    with open(output_file, "w") as f:
        for image in os.listdir(image_dir):
            if image.lower().endswith(('.png', '.jpg', '.jpeg')):
                f.write(os.path.join(image_dir, image) + "\n")
    
    print(f"Created {output_file}")

create_image_list_file("train")
create_image_list_file("val")


Created /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/data/processed_data/train.txt
Created /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/data/processed_data/val.txt


In [6]:
# 2.2 Create rumex_acetosa.data file
data_file_content = f"""classes = 1
train = {os.path.join(DATA_PATH, "train.txt")}
valid = {os.path.join(DATA_PATH, "val.txt")}
names = {os.path.join(DATA_PATH, "rumex_acetosa.names")}
backup = {os.path.join(DARKNET_PATH, "backup")}
"""

with open(os.path.join(DATA_PATH, "rumex_acetosa.data"), "w") as f:
    f.write(data_file_content)

print("Created rumex_acetosa.data")

# 2.3 Create rumex_acetosa.names file
with open(os.path.join(DATA_PATH, "rumex_acetosa.names"), "w") as f:
    f.write("rumex_acetosa\n")

print("Created rumex_acetosa.names")

Created rumex_acetosa.data
Created rumex_acetosa.names


In [7]:
# 2.4 Create yolov3-tiny-rumex.cfg file
subprocess.run(f"cp {os.path.join(DARKNET_PATH, 'cfg', 'yolov3-tiny.cfg')} {os.path.join(DARKNET_PATH, 'cfg', 'yolov3-tiny-rumex.cfg')}", shell=True)

cfg_path = os.path.join(DARKNET_PATH, "cfg", "yolov3-tiny-rumex.cfg")
with open(cfg_path, "r") as f:
    cfg_content = f.read()

cfg_content = cfg_content.replace("classes=80", "classes=1")
cfg_content = cfg_content.replace("filters=255", "filters=18")

with open(cfg_path, "w") as f:
    f.write(cfg_content)

print("Created and modified yolov3-tiny-rumex.cfg")

Created and modified yolov3-tiny-rumex.cfg


In [8]:
# 3. Download pre-trained weights
weights_path = os.path.join(DARKNET_PATH, "yolov3-tiny.weights")
if not os.path.exists(weights_path):
    subprocess.run(f"wget https://pjreddie.com/media/files/yolov3-tiny.weights -O {weights_path}", shell=True)
    print("Downloaded pre-trained weights")
else:
    print("Pre-trained weights already exist")

Pre-trained weights already exist


In [10]:
# 4. Prepare training command
train_command = f"""
cd {DARKNET_PATH} && ./darknet detector train \
    {os.path.join(DATA_PATH, "rumex_acetosa.data")} \
    {os.path.join(DARKNET_PATH, "cfg", "yolov3-tiny-rumex.cfg")} \
    {weights_path} \
    -gpu 0
"""

print("\nTraining command prepared. To start training, run:")
print(train_command)


Training command prepared. To start training, run:

cd /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet && ./darknet detector train     /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/data/processed_data/rumex_acetosa.data     /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet/cfg/yolov3-tiny-rumex.cfg     /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet/yolov3-tiny.weights     -gpu 0



In [11]:
# Execute the training command
print("\nStarting training...")
subprocess.run(train_command, shell=True)



Starting training...


 CUDA-version: 12000 (12060), cuDNN: 9.4.0, GPU count: 1  
 OpenCV version: 4.6.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: NVIDIA GeForce GTX 1660 Ti 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max                2x 2/ 2     26 x  26 x 256

yolov3-tiny-rumex
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 1 
Create CUDA-stream - 0 
 Create cudnn-handle 0 


Done! Loaded 24 layers from weights-file 
Darknet error location: ./src/detector.c, train_detector(), line #91
Error: You set incorrect value batch=1 for Training! You should set batch=64 subdivision=64: Success
backtrace (8 entries)
1/8: ./darknet(log_backtrace+0x38) [0x6493f3b3b6d8]
2/8: ./darknet(error+0x3d) [0x6493f3b3b7bd]
3/8: ./darknet(train_detector+0x4b7e) [0x6493f3be972e]
4/8: ./darknet(run_detector+0x9c0) [0x6493f3bf00c0]
5/8: ./darknet(main+0x324) [0x6493f3af88d4]
6/8: /usr/lib/x86_64-linux-gnu/libc.so.6(+0x2a1ca) [0x72bf4382a1ca]
7/8: /usr/lib/x86_64-linux-gnu/libc.so.6(__libc_start_main+0x8b) [0x72bf4382a28b]
8/8: ./darknet(_start+0x25) [0x6493f3afab55]



 seen 64, trained: 32013 K-images (500 Kilo-batches_64) 


CompletedProcess(args='\ncd /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet && ./darknet detector train     /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/data/processed_data/rumex_acetosa.data     /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet/cfg/yolov3-tiny-rumex.cfg     /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet/yolov3-tiny.weights     -gpu 0\n', returncode=1)

In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0
